In [59]:
## ROBT 310 - Final Project - Virtual Keyboard
## Team: Daryn Kalym, Alibek Manabayev, Alexey Muryshkin
## Date: April 27, 2018

%matplotlib inline

## modules
import numpy as np
from math import sqrt
import cv2
import sys

## global variables
frame = None
background = None

## functions

# Useless Function
def q(*x, **y):
    pass

# Noise Filtering
def noiseFiltering(frame):
    res = grayImage = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    res = medFiltImage = cv2.medianBlur(grayImage, 5)
#     res = cv2.equalizeHist(medFiltImage)
    
    return res

def processFrame(frame):
    global background
    
    threshold = 2500
    
    frame = noiseFiltering(frame)
    mask = np.abs(np.int32(frame)**2 - np.int32(background)**2) > threshold
    backSubImage = np.zeros(frame.shape)
    backSubImage[mask] = 255
    
#     background = 0.7 * background + 0.3 * frame
    kernel = np.ones((5,5),np.uint8)
    erosion = cv2.erode(backSubImage, kernel, iterations = 1)
    kernel = np.ones((15,15),np.uint8)
    dilation = cv2.dilate(erosion, kernel, iterations = 1)
    
    return dilation

def displayVideoRealTime(cap1, cap2):
    global frame, background
    
    cap = cap1
    ret, background = cap.read()
    background = noiseFiltering( np.flip( np.array(background, dtype=np.uint8), axis = 1 ) )
    
    window_name = "WebCam Video Streaming"
    cv2.namedWindow(window_name, flags=cv2.WINDOW_AUTOSIZE)
    cv2.moveWindow(window_name, 0, 0)
    
    while True:        
        ret, frame = cap.read()
        if not ret:
            break
            
        frame = np.flip( np.array(frame, dtype=np.uint8), axis = 1 )
        I2 = processFrame(frame)
        
        cv2.imshow(window_name, np.uint8( I2 ))
        
        key = cv2.waitKey(30)
        c = chr(key & 255)
        if c in ['q', 'Q', chr(27)]:
            break
    
    cap1.release()
    #cap2.release()
    cv2.destroyAllWindows()
    

def getVideoCapture(idd):
    if isinstance(idd, int):
        cap = cv2.VideoCapture( idd )
    else:
        cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print('Cannot initialize video capture with id {}'.format(idd))
        #sys.exit(-1)
        
    return cap

    
## main program
def main():
    id1 = int( input('Please enter the id of the opened video capturing device #1:\n') )
    cap1 = getVideoCapture(id1)
#     id2 = int( input('Please enter the id of the opened video capturing device #2:\n') )
#     cap2 = getVideoCapture(id2)
    cap2 = None
    
    displayVideoRealTime(cap1, cap2)

    
if __name__ == '__main__':
    main()


Please enter the id of the opened video capturing device #1:
1
